# Extraer datos de demanda eléctrica
La ejecución de este código utiliza los datos de los informes mensuales de datos operativos eléctricos de la Compañía Administradora del Mercado Mayorista Eléctrico (CAMMESA) de Argentina, y los agrupa en un único archivo CSV y filtra duplicados. <br><br>
Dentro de los mismos consta un resumen mensual que incluye detalles de demanda (MWh) de grandes usuarios y distribuidoras, por región, provincia, tarifa y categoría.

In [ ]:
# Cargar librerías
import zipfile
import numpy as np
import pandas as pd
from datetime import date, datetime
from dateutil.rrule import rrule, MONTHLY
from os import remove, path

In [ ]:
# Definir fechas
sdate = date(2016, 1, 1)   # fecha de inicio (año, mes, día)
edate = date(2016, 5, 1)   # fecha de fin (año, mes, día)

In [ ]:
# Definir valores a extraer
filePath = './CAMMESA'
fileName = 'BASE_INFORME_MENSUAL_'
sheetName = 'DEMANDA'
headerRow = 21 #should be one less than the row in excel
columns = 'A:L'
columnNames = ['AÑO', 'MES', 'AGENTE_NEMO', 'AGENTE_DESCRIPCION', 'TIPO_AGENTE', 
               'REGION', 'PROVINCIA', 'CATEGORIA_AREA', 'CATEGORIA_DEMANDA', 'TARIFA',
               'CATEGORIA_TARIFA', 'DEMANDA']

outFile = 'Demanda.csv' #nombre del archivo de destino

In [ ]:
#Funciones de fecha
def pad_date(month_num):
    month = str(month_num)
    if len(month) == 1:
        month = '0' + month
    return month

def month_iter(start_month, start_year, end_month, end_year):
    start = datetime(start_year, start_month, 1)
    end = datetime(end_year, end_month, 1)

    return ((pad_date(d.month), pad_date(d.year)) for d in rrule(MONTHLY, dtstart=start, until=end))

In [ ]:
# Descomprimir datos
for (month, year) in month_iter(sdate.month, sdate.year, edate.month, edate.year):
    nameComponents = [year, month]  
    inFile = filePath + '/' + fileName + '-'.join(nameComponents) + '.zip'

    if path.isfile(inFile):
        print('Extracting ' + inFile)
        with zipfile.ZipFile(inFile) as zf:
            zf.extractall(filePath)
    else:
        print('ERROR: ' + inFile + " does not exist")

In [ ]:
# Borrar archivos zip
for (month, year) in month_iter(sdate.month, sdate.year, edate.month, edate.year):
    nameComponents = [year, month]  
    inFile = filePath + '/' + fileName + '-'.join(nameComponents) + '.zip'
    
    if path.isfile(inFile):
        print('Deleting ' + inFile)
        remove(inFile)
    else:
        print('ERROR: ' + inFile + " does not exist")

In [7]:
# Unificar datos en un DataFrame
out = pd.DataFrame()

for (month,year) in month_iter(sdate.month, sdate.year, edate.month, edate.year):
    nameComponents = [year, month]  
    inFile = filePath + '/' + fileName + '-'.join(nameComponents) + '.xlsx'
    
    if path.isfile(inFile):
        print('Concatenating ' + inFile)
        df = pd.read_excel(inFile, sheet_name = sheetName, header = headerRow, usecols = columns)
        df.columns = columnNames
        out = pd.concat([out, df])
    else:
        print('ERROR: ' + inFile + " does not exist")

Concatenating ./CAMMESA/BASE_INFORME_MENSUAL_2016-02.xlsx
Concatenating ./CAMMESA/BASE_INFORME_MENSUAL_2016-03.xlsx
Concatenating ./CAMMESA/BASE_INFORME_MENSUAL_2016-04.xlsx
Concatenating ./CAMMESA/BASE_INFORME_MENSUAL_2016-05.xlsx


In [8]:
out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221267 entries, 0 to 46548
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   AÑO                 221267 non-null  int64         
 1   MES                 221267 non-null  datetime64[ns]
 2   AGENTE_NEMO         221267 non-null  object        
 3   AGENTE_DESCRIPCION  221267 non-null  object        
 4   TIPO_AGENTE         221267 non-null  object        
 5   REGION              221267 non-null  object        
 6   PROVINCIA           221267 non-null  object        
 7   CATEGORIA_AREA      221267 non-null  object        
 8   CATEGORIA_DEMANDA   221267 non-null  object        
 9   TARIFA              221267 non-null  object        
 10  CATEGORIA_TARIFA    221267 non-null  object        
 11  DEMANDA             221267 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(9)
memory usage: 21.9+ MB


In [9]:
#Drop duplicated rows
df = out.drop_duplicates(subset = columnNames, inplace = False)
df.reset_index(drop= True, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53615 entries, 0 to 53614
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   AÑO                 53615 non-null  int64         
 1   MES                 53615 non-null  datetime64[ns]
 2   AGENTE_NEMO         53615 non-null  object        
 3   AGENTE_DESCRIPCION  53615 non-null  object        
 4   TIPO_AGENTE         53615 non-null  object        
 5   REGION              53615 non-null  object        
 6   PROVINCIA           53615 non-null  object        
 7   CATEGORIA_AREA      53615 non-null  object        
 8   CATEGORIA_DEMANDA   53615 non-null  object        
 9   TARIFA              53615 non-null  object        
 10  CATEGORIA_TARIFA    53615 non-null  object        
 11  DEMANDA             53615 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(9)
memory usage: 4.9+ MB


In [10]:
#Exporta la tabla
df.to_csv(outFile, index = False) 